In [ ]:
#@markdown # One Click AUTOMATIC 1111 - Protogen v2.2
from IPython.utils import capture
from IPython.display import clear_output
from subprocess import getoutput
import ipywidgets as widgets
import sys
import fileinput
import os
import time

#@markdown Este notebook se basa en [fast_stable_diffusion_AUTOMATIC1111](https://github.com/TheLastBen/fast-stable-diffusion) creado por [TheLastBen](https://github.com/TheLastBen) y usa el modelo [Protogen V2.2](https://civitai.com/models/3627/protogen-v22-official-release) de [darkstorm2150](https://civitai.com/user/darkstorm2150).

#@markdown Utiliza la Trigger Word **"modelshoot style"** en tus prompts para obtener mejores resultados.


#@markdown <p align="center"><img src="https://github.com/YufniCastro/One-Click-AUTOMATIC-1111-Protogen-v2.2/blob/main/prompt_matrix-0002-487529710.png?raw=true"  width="528" height="414"></p>

#@markdown Prompt: anime character | modelshoot style | blue hair

#@markdown Seed: 487529710

#@markdown ## Intrucciones

#@markdown 1. Añade un usuario y contraseña para controlar el acceso.
User = "" #@param {type:"string"}
Password= "" #@param {type:"string"}

#@markdown 2. Ejecuta esta celda y espera a que se instalen los requerimientos

#@markdown 3. Click en el enlace que aparecera al terminar la instalación. Ingresa con tu usuario y contraseña (ejemplo ↓):

#@markdown > Running on public URL: https://xxxxxxxxxxxxxxxxxxxxx.gradio.live

#@markdown 4. **Todo listo!** Comienza a generar


#@markdown [Repositorio](https://github.com/YufniCastro/One-Click-AUTOMATIC-1111-Protogen-v2.2) <- reportar errores

#@markdown Creado por [Yufni](https://github.com/YufniCastro)

!mkdir -p /content/gdrive/MyDrive/
with capture.capture_output() as cap:
  def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)
  fgitclone = "git clone --depth 1"
  %mkdir -p /content/gdrive/MyDrive/sd
  %cd /content/gdrive/MyDrive/sd
  !$fgitclone -q --branch master https://github.com/AUTOMATIC1111/stable-diffusion-webui
  !mkdir -p /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface
  !ln -s /content/gdrive/MyDrive/sd/stable-diffusion-webui/cache/huggingface /root/.cache/

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/
  !git reset --hard
  time.sleep(1)
  !rm webui.sh
print('[1;32m')
!git pull
clear_output()
inf('\u2714 Done','success', '50px')

print('[1;32mInstalling requirements...')

with capture.capture_output() as cap:
  %cd /content/ 
  !wget -q -i https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/Dependencies/A1111.txt
  !dpkg -i *.deb
  if not os.path.exists('/content/gdrive/MyDrive/sd/stablediffusion'):
    !tar -C / --zstd -xf sd.tar.zst 
  !tar -C / --zstd -xf A1111_dep.tar.zst
  !rm *.deb | rm *.zst | rm *.txt
  %env LD_PRELOAD=libtcmalloc.so
  os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

clear_output()
inf('\u2714 Done','success', '50px')



MODEL_LINK = "https://civitai.com/api/download/models/4007?type=Model&format=SafeTensor"
safetensors = False
Use_temp_storage = False 



gdrv="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion"
tmp="/content"
pth=tmp if Use_temp_storage else gdrv
%cd $pth
clear_output()
if not safetensors:
  modelname="model.ckpt"
else:
  modelname="model.safetensors"
!gdown --fuzzy -O $modelname $MODEL_LINK
if os.path.exists(f'{pth}/{modelname}') and os.path.getsize(f'{pth}/{modelname}') > 1810671599:    
    model=f'{pth}/{modelname}'
    clear_output()
    inf('\u2714 Model downloaded, using the trained model.','success', '350px')
else:
  if Use_temp_storage:
    !rm $pth/$modelname
  else:
    rmv()
  inf('\u2718 Wrong link, check that the link is valid','danger', "300px")

# Start Stable-Diffusion
from IPython.utils import capture
import time
import sys
import fileinput

Use_localtunnel = False

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

with capture.capture_output() as cap:
  if not os.path.exists('/tools/node/bin/lt'):
    !npm install -g localtunnel

with capture.capture_output() as cap:
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/
  !wget -O paths.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/paths.py
  !wget -O extras.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/extras.py
  !wget -O sd_models.py https://raw.githubusercontent.com/AUTOMATIC1111/stable-diffusion-webui/master/modules/sd_models.py
  !wget -q -O /usr/local/lib/python3.8/dist-packages/gradio/blocks.py https://raw.githubusercontent.com/TheLastBen/fast-stable-diffusion/main/AUTOMATIC1111_files/blocks.py
  %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui/

  !sed -i "s@os.path.splitext(checkpoint_file)@os.path.splitext(checkpoint_file); map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/sd_models.py
  !sed -i 's@ui.create_ui().*@ui.create_ui();shared.demo.queue(concurrency_count=999999,status_update_rate=0.1)@' /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py
  !sed -i "s@map_location='cpu'@map_location='cuda'@" /content/gdrive/MyDrive/sd/stable-diffusion-webui/modules/extras.py

share=''
if not Use_localtunnel:
  share='--share'

else:
  with capture.capture_output() as cap:
    share=''
    %cd /content
    !nohup lt --port 7860 > srv.txt 2>&1 &
    time.sleep(2)
    !grep -o 'https[^ ]*' /content/srv.txt >srvr.txt
    time.sleep(2)
    srv= getoutput('cat /content/srvr.txt')

    for line in fileinput.input('/usr/local/lib/python3.8/dist-packages/gradio/blocks.py', inplace=True):
      if line.strip().startswith('self.server_name ='):
          line = f'            self.server_name = "{srv[8:]}"\n'
      if line.strip().startswith('self.protocol = "https"'):
          line = '            self.protocol = "https"\n'
      if line.strip().startswith('if self.local_url.startswith("https") or self.is_colab'):
          line = ''
      if line.strip().startswith('else "http"'):
          line = ''
      sys.stdout.write(line)
            
    !rm /content/srv.txt /content/srvr.txt
    %cd /content/gdrive/MyDrive/sd/stable-diffusion-webui

try:
  model
  if os.path.isfile(model):
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt "$model" --xformers $auth --disable-console-progressbars
  else:
    !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae  --ckpt-dir "$model" --xformers $auth --disable-console-progressbars
except:
   !python /content/gdrive/MyDrive/sd/stable-diffusion-webui/webui.py $share --api --disable-safe-unpickle --enable-insecure-extension-access --no-download-sd-model --no-half-vae --xformers $auth --disable-console-progressbars